In [6]:
# imports 
import pandas as pd
import requests
import zipfile
import io
import numpy as np
import folium
import geocoder

In [2]:
# 2013 DC crime data
link = 'http://data.octo.dc.gov/feeds/crime_incidents/archive/crime_incidents_2013_CSV.zip'
r2013 = requests.get(link)
z = zipfile.ZipFile(io.BytesIO(r2013.content))
df = pd.read_csv(z.open('crime_incidents_2013_CSV.csv'))

In [3]:
# drop missing voting_precinct values
df = df[pd.notnull(df['VOTING_PRECINCT'])]

# create a more complete address column
df['ADDRESS'] = df['BLOCKSITEADDRESS'] + ', WASHINGTON, DC'
df['ADDRESS'] = df['ADDRESS'].map(lambda x: np.nan if '-' not in x else x)
df = df[pd.notnull(df['ADDRESS'])]
df['ADDRESS'] = df['ADDRESS'].map(lambda x: x.split('-')[1][1:])
df['ADDRESS'] = df['ADDRESS'].map(lambda x: x.replace('BLOCK OF ', ''))

In [4]:
# retain only violent crimes
violent = ['ASSAULT W/DANGEROUS WEAPON', 'HOMICIDE', 'SEX ABUSE']
df_violent = df[df['OFFENSE'].isin(violent)]
print(df_violent.shape)

(2549, 21)

In [7]:
# geocode the addresses
def f_geo_lat(x):
    geocode_result = geocoder.bing(x, key = bing_key)
    try:
        return geocode_result.json['lat']
    except:
        return np.nan

def f_geo_lon(x):
    geocode_result = geocoder.bing(x, key = bing_key)
    try:
        return geocode_result.json['lng']
    except:
        return np.nan
    
bing_key = 'bingapikey'
df_violent['LAT'] = df_violent['ADDRESS'].map(f_geo_lat)
df_violent['LON'] = df_violent['ADDRESS'].map(f_geo_lon)

C:\Users\Michael\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Michael\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [30]:
# calculate the number of incidents per district
df_voting = df_violent['VOTING_PRECINCT'].value_counts().astype(float)
df_voting.to_json('votingagg.json')
df_voting = df_voting.reset_index()
df_voting.columns = ['VOTING_PRECINCT','AMOUNT']

# ensure that precincts with values of zero are included in the dataframe
pre = df_voting['VOTING_PRECINCT'].unique()
pl = df_voting.index[-1] + 1
for d in df['VOTING_PRECINCT'].unique():
    if d not in pre:
        df_voting.loc[pl] = [d, 0]
        pl += 1

# check that all precincts are included
df_voting.shape

(143, 2)

In [40]:
# defined boundaries of voting precincts
geo = 'voting-precinct-2012.geojson'

# create choropleth map
DC_COORD = (38.9072, -77.0369)
map_v = folium.Map(location= DC_COORD, zoom_start=11)
map_v.choropleth(geo_path = geo,
                 data = df_voting,
                 columns = ['VOTING_PRECINCT','AMOUNT'],
                 key_on = 'feature.properties.NAME',
                 fill_color = 'OrRd',
                 legend_name = 'Number of Violent')
map_v

C:\Users\Michael\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.


In [39]:
df_violent.groupby('OFFENSE')['ANC'].count().reset_index()

,OFFENSE,ANC
0,ASSAULT W/DANGEROUS WEAPON,2181
1,HOMICIDE,99
2,SEX ABUSE,269
